In [176]:
import io
import boto3
import sagemaker
import json
from sagemaker import get_execution_role
import os
from sklearn.datasets import *
import pandas as pd
from botocore.exceptions import ClientError
import awswrangler as wr
from datetime import date

In [177]:
# Get region
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()  # replace with your own bucket name if you have one
role = 'sagemaker_developer'
prefix = "data/tabular/california_housing"
filename = "california_housing.csv"

In [178]:
iam = boto3.client("iam")
sts = boto3.client("sts")
redshift = boto3.client("redshift")
sm = boto3.client("sagemaker")
s3 = sagemaker_session.boto_session.resource("s3")


In [179]:
role_name = role.split("/")[-1]
print("Your Role name used to create this notebook is: {}".format(role_name))

Your Role name used to create this notebook is: sagemaker_developer


Download data from online resources and write data to S3

In [180]:
# helper functions to upload data to s3
def write_to_s3(filename, bucket, prefix):
    # put one file in a separate folder. This is helpful if you read and prepare data with Athena
    filename_key = filename.split(".")[0]
    key = "{}/{}/{}".format(prefix, filename_key, filename)
    return s3.Bucket(bucket).upload_file(filename, key)


def upload_to_s3(bucket, prefix, filename):
    url = "s3://{}/{}/{}".format(bucket, prefix, filename)
    print("Writing to {}".format(url))
    write_to_s3(filename, bucket, prefix)

In [181]:
tabular_data = fetch_california_housing()
tabular_data_full = pd.DataFrame(tabular_data.data, columns=tabular_data.feature_names)
tabular_data_full["target"] = pd.DataFrame(tabular_data.target)
tabular_data_full.to_csv("california_housing.csv", index=False)

upload_to_s3(bucket, "data/tabular", filename)

Writing to s3://sagemaker-us-east-1-083839308414/data/tabular/california_housing.csv


Create Redshift Role
The policy enables Redshift to assume the role. The services can then perform any tasks granted by the permissions policy assigned to the role (which we will attach to it later).

In [182]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "redshift.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

In [183]:
# Create Role
iam_redshift_role_name = "Tabular_Redshift"
try:
    iam_role_redshift = iam.create_role(
        RoleName=iam_redshift_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description="Tabular data Redshift Role",
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


In [184]:
# get role arn
role_rs = iam.get_role(RoleName="Tabular_Redshift")
iam_role_redshift_arn = role_rs["Role"]["Arn"]
print("Your Role arn used to create a Redshift Cluster is: {}".format(iam_role_redshift_arn))

Your Role arn used to create a Redshift Cluster is: arn:aws:iam::083839308414:role/Tabular_Redshift


Create Policy Document
We will create policies we used to access S3 and Athena. The two policies we will create here are:

S3FullAccess: arn:aws:iam::aws:policy/AmazonS3FullAccess
AthenaFullAccess: arn:aws:iam::aws:policy/AmazonAthenaFullAccess
You can check the policy document in the IAM console and copy the policy file here.

In [185]:
# s3FullAccess
my_redshift_to_s3 = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Action": "s3:*", "Resource": "*"}],
}

In [186]:
# Athena Full Access
my_redshift_to_athena = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["athena:*"], "Resource": ["*"]},
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition",
            ],
            "Resource": ["*"],
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload",
                "s3:CreateBucket",
                "s3:PutObject",
            ],
            "Resource": ["arn:aws:s3:::aws-athena-query-results-*"],
        },
        {
            "Effect": "Allow",
            "Action": ["s3:GetObject", "s3:ListBucket"],
            "Resource": ["arn:aws:s3:::athena-examples*"],
        },
        {
            "Effect": "Allow",
            "Action": ["s3:ListBucket", "s3:GetBucketLocation", "s3:ListAllMyBuckets"],
            "Resource": ["*"],
        },
        {
            "Effect": "Allow",
            "Action": ["sns:ListTopics", "sns:GetTopicAttributes"],
            "Resource": ["*"],
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricAlarm",
                "cloudwatch:DescribeAlarms",
                "cloudwatch:DeleteAlarms",
            ],
            "Resource": ["*"],
        },
        {"Effect": "Allow", "Action": ["lakeformation:GetDataAccess"], "Resource": ["*"]},
    ],
}

In [187]:
try:
    policy_redshift_s3 = iam.create_policy(
        PolicyName="Tabular_RedshiftPolicyToS3", PolicyDocument=json.dumps(my_redshift_to_s3)
    )
    print("Policy created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

account_id = sts.get_caller_identity()["Account"]
policy_redshift_s3_arn = f"arn:aws:iam::{account_id}:policy/Tabular_RedshiftPolicyToS3"

Policy already exists


In [188]:
try:
    policy_redshift_athena = iam.create_policy(
        PolicyName="Tabular_RedshiftPolicyToAthena",
        PolicyDocument=json.dumps(my_redshift_to_athena),
    )
    print("Policy created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

account_id = sts.get_caller_identity()["Account"]
policy_redshift_athena_arn = f"arn:aws:iam::{account_id}:policy/Tabular_RedshiftPolicyToAthena"

Policy already exists


In [189]:
# Attach RedshiftPolicyToAthena policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_athena_arn, RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [190]:
# Attach RedshiftPolicyToS3 policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_s3_arn, RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Making Sure your Role to run this Notebook has the following policy attached:
SecretsManagerReadWrite: we will use this service to store and retrive our Redshift Credentials.
AmazonRedshiftFullAccess: we will use this role to create a Redshift cluster from the notebook.

In [191]:
# making sure you have secret manager policy attached to role
try:
    policy = "SecretsManagerReadWrite"
    response = iam.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name
    )
    print("Policy %s has been succesfully attached to role: %s" % (policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy is already attached.")
    else:
        print("Unexpected error: %s " % e)

Policy SecretsManagerReadWrite has been succesfully attached to role: sagemaker_developer


In [192]:
# making sure you have RedshiftFullAccess policy attached to role
from botocore.exceptions import ClientError

try:
    policy = "AmazonRedshiftFullAccess"
    response = iam.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name
    )
    print("Policy %s has been succesfully attached to role: %s" % (policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy is already attached. ")
    else:
        print("Unexpected error: %s " % e)

Policy AmazonRedshiftFullAccess has been succesfully attached to role: sagemaker_developer


Optional: Create Redshift Cluster
Most of the times we have a Redshift cluster already up and running and we want to connect to the cluster in-use, but if you want to create a new cluster, you can follow the steps below to create one. Note that only some Instance Types support Redshift Query Editor, so be careful when you specify the Redshift Cluster Nodes.(https://docs.aws.amazon.com/redshift/latest/mgmt/query-editor.html).

In [193]:
notebook_instance_name = sm.list_notebook_instances()["NotebookInstances"][0][
    "NotebookInstanceName"
]
print(notebook_instance_name)

IndexError: list index out of range

Create Secret in Secrets Manager
Your IAM role will need permission to create a secret and get its value. This can be accomplished with the SecretsManagerReadWrite managed policy.

AWS Secrets Manager is a service that enables you to easily rotate, manage, and retrieve database credentials, API keys, and other secrets throughout their lifecycle. Using Secrets Manager, you can secure and manage secrets used to access resources in the AWS Cloud, on third-party services, and on-premises.

*note that MasterUserPassword must contain at least 1 upper case letter and at least 1 decimal digit.

Ensure that you change the secret password to be unique and secure.

In [ ]:
secretsmanager = boto3.client("secretsmanager")

try:
    response = secretsmanager.create_secret(
        Name="tabular_redshift_login",
        Description="California Housing data New Cluster Redshift Login",
        SecretString='[{"username":"awsuser"},{"password":"Californiahousing1"}]',
        Tags=[
            {"Key": "name", "Value": "tabular_redshift_login"},
        ],
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceExistsException":
        print("Secret already exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [ ]:
# And retrieving the secret again
secretsmanager = boto3.client("secretsmanager")
import json

secret = secretsmanager.get_secret_value(SecretId="tabular_redshift_login")
cred = json.loads(secret["SecretString"])

master_user_name = cred[0]["username"]
master_user_pw = cred[1]["password"]

In [ ]:
# Set up parameters
# Redshift configuration parameters
redshift_cluster_identifier = "redshiftdemo"
database_name = "california_housing"
cluster_type = "multi-node"
node_type = "dc2.large"
number_nodes = "2"

In [ ]:
notebook_instance = sm.describe_notebook_instance(NotebookInstanceName=notebook_instance_name)
security_group_id = notebook_instance["SecurityGroups"][0]
print(security_group_id)

ParamValidationError: Parameter validation failed:
Invalid type for parameter NotebookInstanceName, value: {'NotebookInstances': [], 'ResponseMetadata': {'RequestId': '74d33a4a-8153-4c57-b7fe-a83fdcb3b387', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '74d33a4a-8153-4c57-b7fe-a83fdcb3b387', 'content-type': 'application/x-amz-json-1.1', 'content-length': '24', 'date': 'Tue, 31 Jan 2023 09:08:00 GMT'}, 'RetryAttempts': 0}}, type: <class 'dict'>, valid types: <class 'str'>

In [ ]:
response = redshift.create_cluster(
    DBName=database_name,
    ClusterIdentifier=redshift_cluster_identifier,
    ClusterType=cluster_type,
    NodeType=node_type,
    NumberOfNodes=int(number_nodes),
    MasterUsername=master_user_name,
    MasterUserPassword=master_user_pw,
    # ClusterSubnetGroupName="<cluster-subnet-group-1>",  # you can either specify an existing subnet group (change this to your Subnet Group name), or use the security group ID that was retrieved above
    IamRoles=[iam_role_redshift_arn],
    VpcSecurityGroupIds=[security_group_id],
    Port=5439,
    PubliclyAccessible=False,
)

print(response)

NameError: name 'security_group_id' is not defined

In [ ]:
# check cluster status
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response["Clusters"][0]["ClusterStatus"]
print("Your Redshift Cluster Status is: " + cluster_status)

In [200]:
# Get Redshift Endpoint Address & IAM Role
redshift_endpoint_address = response["Clusters"][0]["Endpoint"]["Address"]
iam_role ='demo-iam-role'

print("Redshift endpoint: {}".format(redshift_endpoint_address))
print("IAM Role: {}".format(iam_role))

KeyError: 'Clusters'

In [196]:
# Connect to Redshift Database Engine
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        master_user_name,
        master_user_pw,
        redshift_endpoint_address,
        redshift_port,
        database_name_redshift,
    )
)

Create Session: we will use this session to run SQL commands

In [198]:
# config session
session = sessionmaker()
session.configure(bind=engine)
s = session()

Method 1: Access Data without Moving it to Redshift: Amazon Redshift Spectrum
Redshift Spectrum is used to query data directly from files on Amazon S3.You will need to create external tables in an external schema. The external schema references a database in the external data catalog and provides the IAM role ARN that authorizes your cluster to access Amazon S3 on your behalf.

Get table and schema information from the Glue Catalog: getting meta data from data catalog and connecting to the Athena database

In [199]:


statement = """
rollback;
create external schema if not exists {} from data catalog 
    database '{}' 
    iam_role '{}'
    create external database if not exists
""".format(
    schema_spectrum, database_name_athena, iam_role
)

s.execute(statement)
s.commit()

InternalError: (psycopg2.InternalError) Unknown std exception when calling external catalog API: 
  -----------------------------------------------
  error:  exception name : UnauthorizedException, error type : 135, message: Not authorized to get credentials of role arn:aws:iam::083839308414:role/demo-iam-role, should retry : 0
  code:      30000
  context:   
  query:     0
  location:  xen_aws_credentials_mgr.cpp:411
  process:   padbmaster [pid=6708]
  -----------------------------------------------


[SQL: 
rollback;
create external schema if not exists spectrum from data catalog 
    database 'tabular_california_housing' 
    iam_role 'arn:aws:iam::083839308414:role/demo-iam-role'
    create external database if not exists
]
(Background on this error at: http://sqlalche.me/e/13/2j85)

Run a sample query through Redshift Spectrum

In [ ]:
statement = """
select *
    from {}.{} limit 10
""".format(
    schema_spectrum, table_name_csv
)

df = pd.read_sql_query(statement, engine)
df.head(5)